In [1]:
import git
!git clone https://github.com/TakHemlata/SSL_Anti-spoofing

Cloning into 'SSL_Anti-spoofing'...
remote: Enumerating objects: 1579, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 1579 (delta 52), reused 82 (delta 48), pack-reused 1489 (from 1)
Receiving objects: 100% (1579/1579), 30.57 MiB | 18.91 MiB/s, done.
Resolving deltas: 100% (293/293), done.


In [ ]:
import shutil
import os

source_dir = '/kaggle/working/SSL_Anti-spoofing'
destination_dir = '/kaggle/working/'

os.makedirs(destination_dir, exist_ok=True)

for item_name in os.listdir(source_dir):
    source_item = os.path.join(source_dir, item_name)
    destination_item = os.path.join(destination_dir, item_name)
    
    shutil.move(source_item, destination_item)

print(f'All files and folders moved from {source_dir} to {destination_dir}')

All files and folders moved from /kaggle/working/SSL_Anti-spoofing to /kaggle/working/


In [ ]:
file_path = "/kaggle/working/fairseq-a54021305d6b3c4c5959ac9395135f63202db8f1/fairseq/data/indexed_dataset.py"

try:
    os.remove(file_path)
    print(f"File {file_path} deleted successfully.")
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")
except PermissionError:
    print("You do not have permission to delete this file.")

File /kaggle/working/fairseq-a54021305d6b3c4c5959ac9395135f63202db8f1/fairseq/data/indexed_dataset.py deleted successfully.


In [ ]:
source_path = "/kaggle/input/float32/indexed_dataset.py"
destination_path = "/kaggle/working/fairseq-a54021305d6b3c4c5959ac9395135f63202db8f1/fairseq/data/indexed_dataset.py"

shutil.copy2(source_path, destination_path)

print(f'Copied {source_path} to {destination_path}')

Copied /kaggle/input/float32/indexed_dataset.py to /kaggle/working/fairseq-a54021305d6b3c4c5959ac9395135f63202db8f1/fairseq/data/indexed_dataset.py


In [5]:
!cd fairseq-a54021305d6b3c4c5959ac9395135f63202db8f1 && pip install --editable ./

Obtaining file:///kaggle/working/fairseq-a54021305d6b3c4c5959ac9395135f63202db8f1
  Installing build dependencies ... - \ | / - done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... - \ | / done
  Preparing editable metadata (pyproject.toml) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 17.1 MB/s eta 0:00:00
  Building editable for fairseq (pyproject.toml) ... - \ | / - \ | / - \ done
  Created wheel for fairseq: filename=fairseq-1.0.0a0+4acaa61-0.editable-cp310

In [6]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.2 MB/s eta 0:00:00
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: tensorboardX
    Found existing installation: tensorboardX 2.6.2.2
    Uninstalling tensorboardX-2.6.2.2:
      Successfully uninstalled tensorboardX-2.6.2.2
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.2.post1
    Uninstalling librosa-0.10.2.post1:
      Successfully uninstalled librosa-0.10.2.post1


In [ ]:
import sys
sys.path.append('/kaggle/working/fairseq-a54021305d6b3c4c5959ac9395135f63202db8f1')

import os
import numpy as np
import torch
from torch import nn
from torch import Tensor
from torch.utils.data import DataLoader
import yaml
from data_utils_SSL import genSpoof_list,Dataset_ASVspoof2019_train,Dataset_ASVspoof2021_eval
from model import Model
from tensorboardX import SummaryWriter
from core_scripts.startup_config import set_random_seed

In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.amp import GradScaler, autocast

scaler = GradScaler(device='cuda')

def evaluate_accuracy(dev_loader, model, device, max_samples=None):
    val_loss = 0.0
    num_total = 0
    model.eval()
    criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([0.1, 0.9]).to(device))
    
    with torch.no_grad():
        for batch_x, batch_y in dev_loader:
            batch_size = batch_x.size(0)
            
            if max_samples and num_total + batch_size > max_samples:
                batch_size = int(max_samples - num_total)
                batch_x = batch_x[:batch_size]
                batch_y = batch_y[:batch_size]
            
            num_total += batch_size
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.view(-1).type(torch.int64).to(device)
            
            batch_out = model(batch_x)
            batch_loss = criterion(batch_out, batch_y)
            
            val_loss += (batch_loss.item() * batch_size)
            
            if max_samples and num_total >= max_samples:
                break
    
    val_loss /= num_total
    return val_loss

def train_epoch(train_loader, model, optimizer, device):
    running_loss = 0.0
    num_total = 0.0
    model.train()
    
    criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([0.1, 0.9]).to(device))
    
    batch_iterator = tqdm(train_loader, desc="Training", leave=True)
    for batch_x, batch_y in batch_iterator:
        batch_size = batch_x.size(0)
        num_total += batch_size
        
        batch_x = batch_x.to(device, non_blocking=True)
        batch_y = batch_y.view(-1).type(torch.int64).to(device, non_blocking=True)
        
        optimizer.zero_grad()
        
        with autocast('cuda'):
            batch_out = model(batch_x)
            batch_loss = criterion(batch_out, batch_y)
        
        scaler.scale(batch_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += (batch_loss.item() * batch_size)
        
        batch_iterator.set_description(f"Training - Batch Loss: {batch_loss.item():.4f}")
        
    running_loss /= num_total
    return running_loss

In [ ]:
args = {
    'batch_size': 16,
    'num_epochs': 3,
    'lr': 0.000001,
    'weight_decay': 0.0001,
    'loss': 'weighted_CCE',
    'seed': 228,  # Random seed for reproducibility
    'model_path': '/kaggle/input/w2v-airi/models/model_LA_weighted_CCE_3_16_1e-06/epoch_2.pth',  # MODEL FROM MY PREV NOTEBOOK ssl-train-3epochs.ipynb
    'comment': None,  # Description for the saved model
    'track': 'LA',  # Dataset track; options are 'LA', 'PA', 'DF'
    'eval_output': None,  # Path to save evaluation results
    'eval': False,  # Boolean for evaluation mode
    'is_eval': False,  # Boolean for evaluation dataset
    'eval_part': 0,  # Part of evaluation to process
    'cudnn_deterministic_toggle': False,  # Use deterministic CuDNN behavior
    'cudnn_benchmark_toggle': True,  # Use CuDNN benchmark for faster runtime
    
    # RawBoost data augmentation options
    'algo': 4,  # RawBoost algorithm selection

    # LnL_convolutive_noise parameters
    'nBands': 5,  # Number of notch filters
    'minF': 20,  # Minimum center frequency of notch filter
    'maxF': 8000,  # Maximum center frequency of notch filter
    'minBW': 100,  # Minimum bandwidth of filter
    'maxBW': 1000,  # Maximum bandwidth of filter
    'minCoeff': 10,  # Minimum filter coefficients
    'maxCoeff': 100,  # Maximum filter coefficients
    'minG': 0,  # Minimum gain factor of linear component
    'maxG': 0,  # Maximum gain factor of linear component
    'minBiasLinNonLin': 5,  # Minimum gain difference between linear/non-linear components
    'maxBiasLinNonLin': 20,  # Maximum gain difference between linear/non-linear components
    'N_f': 5,  # Order of non-linearity (1 means only linear)

    # ISD_additive_noise parameters
    'P': 10,  # Max number of uniformly distributed samples in [%]
    'g_sd': 2,  # Gain parameter for additive noise

    # SSI_additive_noise parameters
    'SNRmin': 10,  # Minimum SNR for colored noise
    'SNRmax': 40  # Maximum SNR for colored noise
}

from types import SimpleNamespace
args = SimpleNamespace(**args)

In [10]:
if not os.path.exists('models'):
    os.mkdir('models')
set_random_seed(args.seed, args)
track = args.track
assert track in ['LA', 'PA','DF'], 'Invalid track given'

cudnn_deterministic set to False
cudnn_benchmark set to True


In [ ]:
import random
from typing import Union

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import fairseq

class SSLModel(nn.Module):
    def __init__(self, device):
        super(SSLModel, self).__init__()
        
        cp_path = '/kaggle/input/xlsr2-300m/xlsr2_300m.pt'
        model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
        self.model = model[0].to(device)
        self.device = device
        self.out_dim = 1024

    def extract_feat(self, input_data):
        input_data = input_data.to(self.device)
        input_tmp = input_data[:, :, 0] if input_data.ndim == 3 else input_data
        emb = self.model(input_tmp, mask=False, features_only=True)['x']
        
        return emb


class PSFAN_Backend(nn.Module):
    def __init__(self, input_channels=128, num_classes=2):
        super(PSFAN_Backend, self).__init__()
        
        self.conv1 = nn.Conv1d(input_channels, 128, kernel_size=3, dilation=1, padding=1)
        self.conv1x1_1 = nn.Conv1d(128, 128, kernel_size=1)
        self.conv3x3_1 = nn.Conv1d(128, 128, kernel_size=3, padding=1)
        self.conv1x1_2 = nn.Conv1d(128, 128, kernel_size=1)
        self.attention1 = nn.Sigmoid()
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        self.conv2 = nn.Conv1d(128, 128, kernel_size=3, dilation=2, padding=2)
        self.conv1x1_3 = nn.Conv1d(128, 128, kernel_size=1)
        self.conv3x3_2 = nn.Conv1d(128, 128, kernel_size=3, padding=1)
        self.conv1x1_4 = nn.Conv1d(128, 128, kernel_size=1)
        self.attention2 = nn.Sigmoid()
        self.pool2 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, dilation=3, padding=3)
        self.conv1x1_5 = nn.Conv1d(256, 256, kernel_size=1)
        self.conv3x3_3 = nn.Conv1d(256, 256, kernel_size=3, padding=1)
        self.conv1x1_6 = nn.Conv1d(256, 256, kernel_size=1)
        self.attention3 = nn.Sigmoid()
        self.pool3 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.conv4 = nn.Conv1d(256, 256, kernel_size=3, dilation=4, padding=4)
        self.conv1x1_7 = nn.Conv1d(256, 256, kernel_size=1)
        self.conv3x3_4 = nn.Conv1d(256, 256, kernel_size=3, padding=1)
        self.conv1x1_8 = nn.Conv1d(256, 256, kernel_size=1)
        self.attention4 = nn.Sigmoid()
        self.pool4 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.gap1 = nn.AdaptiveAvgPool1d(1)
        self.gap2 = nn.AdaptiveAvgPool1d(1)
        self.gap3 = nn.AdaptiveAvgPool1d(1)
        self.gap4 = nn.AdaptiveAvgPool1d(1)
        
        self.fc_concat = nn.Linear(128 + 128 + 256 + 256, 16)
        self.fc_out = nn.Linear(16, num_classes)
        
        self.activation = nn.LeakyReLU(0.02)

    def forward(self, x):
        x1 = self.conv1(x)
        x1_attention = self.attention1(self.conv1x1_1(self.conv3x3_1(self.conv1x1_2(x1))))
        x1 = x1_attention * x1
        x1 = self.pool1(x1)
        x1_gap = self.gap1(x1).squeeze(-1)

        x2 = self.conv2(x1)
        x2_attention = self.attention2(self.conv1x1_3(self.conv3x3_2(self.conv1x1_4(x2))))
        x2 = x2_attention * x2
        x2 = self.pool2(x2)
        x2_gap = self.gap2(x2).squeeze(-1)

        x3 = self.conv3(x2)
        x3_attention = self.attention3(self.conv1x1_5(self.conv3x3_3(self.conv1x1_6(x3))))
        x3 = x3_attention * x3
        x3 = self.pool3(x3)
        x3_gap = self.gap3(x3).squeeze(-1)

        x4 = self.conv4(x3)
        x4_attention = self.attention4(self.conv1x1_7(self.conv3x3_4(self.conv1x1_8(x4))))
        x4 = x4_attention * x4
        x4 = self.pool4(x4)
        x4_gap = self.gap4(x4).squeeze(-1)

        x_concat = torch.cat([x1_gap, x2_gap, x3_gap, x4_gap], dim=1)

        x = self.activation(self.fc_concat(x_concat))
        output = self.fc_out(x)

        return output


class Model(nn.Module):
    def __init__(self, args, device):
        super(Model, self).__init__()
        self.device = device
        
        self.ssl_model = SSLModel(self.device)
        self.LL = nn.Linear(self.ssl_model.out_dim, 128).to(device)
        self.backend = PSFAN_Backend(input_channels=128, num_classes=2).to(device)

    def forward(self, x):
        x = x.to(self.device)
        x_ssl_feat = self.ssl_model.extract_feat(x)
        x = self.LL(x_ssl_feat)
        x = x.transpose(1, 2)
        output = self.backend(x)
        
        return output

In [ ]:
model_tag = 'model_{}_{}_{}_{}_{}'.format(
    track, args.loss, args.num_epochs, args.batch_size, args.lr)
if args.comment:
    model_tag = model_tag + '_{}'.format(args.comment)
model_save_path = os.path.join('models', model_tag)

if not os.path.exists(model_save_path):
    os.mkdir(model_save_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'                  
print('Device: {}'.format(device))

model = Model(args,device)
nb_params = sum([param.view(-1).size()[0] for param in model.parameters()])
model = model.to(device)
print('nb_params:',nb_params)

optimizer = torch.optim.Adam(model.parameters(), lr=args.lr,weight_decay=args.weight_decay)

if args.model_path:
    model.load_state_dict(torch.load(args.model_path,map_location=device))
    print('Model loaded : {}'.format(args.model_path))

Device: cuda


/kaggle/working/fairseq-a54021305d6b3c4c5959ac9395135f63202db8f1/fairseq/checkpoint_utils.py:313: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_loc

nb_params: 318749618


/tmp/ipykernel_23/3542066152.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.model_path,map_location=device))


Model loaded : /kaggle/input/w2v-airi/models/model_LA_weighted_CCE_3_16_1e-06/epoch_2.pth


In [ ]:
# model_path1 = '/kaggle/input/fed1-dev-and-train/fed1/epoch_2 (1).pth'
# model_path2 = '/kaggle/input/fed1-dev-and-train/fed1/epoch_2.pth'
# model_path3 = '/kaggle/input/fed1-test/epoch_1.pth'

# # i found this weight on kaggle, they gives 0.38 (just pretrain in asv2019 w/o dev&eval)

# model_tag = 'model_{}_{}_{}_{}_{}'.format(
#     track, args.loss, args.num_epochs, args.batch_size, args.lr)
# if args.comment:
#     model_tag = model_tag + '_{}'.format(args.comment)
# model_save_path = os.path.join('models', model_tag)

# if not os.path.exists(model_save_path):
#     os.mkdir(model_save_path)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'                  
# print('Device: {}'.format(device))

# model = Model(args, device).to(device)
# nb_params = sum([param.view(-1).size()[0] for param in model.parameters()])
# print('nb_params:', nb_params)

# def load_model_weights(path, device):
#     if os.path.exists(path):
#         return torch.load(path, map_location=device)
#     else:
#         print(f"Warning: Model path {path} does not exist.")
#         return None

# state_dict1 = load_model_weights(model_path1, device)
# state_dict2 = load_model_weights(model_path2, device)
# state_dict3 = load_model_weights(model_path3, device)

# if state_dict1 and state_dict2 and state_dict3:
#     aggregated_state_dict = {}
#     for key in state_dict1.keys():
#         aggregated_state_dict[key] = (state_dict1[key] + state_dict2[key] + state_dict3[key]) / 3.0
#     del state_dict1, state_dict2, state_dict3
#     model.load_state_dict(aggregated_state_dict)
#     del aggregated_state_dict
#     print("Aggregated model weights loaded successfully.")
# else:
#     print("Error: Failed to load weights from all clients. Check file paths.")

# optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

In [ ]:
d_label_trn, file_train = genSpoof_list(
    dir_meta="/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt",
    is_train=True, is_eval=False)
print('no. of training trials', len(file_train))

train_set = Dataset_ASVspoof2019_train(
    args, list_IDs=file_train, labels=d_label_trn,
    base_dir="/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_train/",
    algo=args.algo)

d_label_dev, file_dev = genSpoof_list(
    dir_meta="/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt",
    is_train=False, is_eval=False)
print('no. of dev trials', len(file_dev))

dev_set = Dataset_ASVspoof2019_train(
    args, list_IDs=file_dev, labels=d_label_dev,
    base_dir="/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_dev/",
    algo=args.algo)

d_label_eval, file_eval = genSpoof_list(
    dir_meta="/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt",
    is_train=False, is_eval=False)
print('no. of validation trials', len(file_dev))

eval_set = Dataset_ASVspoof2019_train(
    args, list_IDs=file_eval, labels=d_label_eval,
    base_dir="/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_eval/",
    algo=args.algo)

no. of training trials 25380
no. of dev trials 24844
no. of validation trials 71237


In [ ]:
from torch.utils.data import ConcatDataset

combined_dataset = ConcatDataset([train_set, dev_set, eval_set])

train_loader = DataLoader(
    combined_dataset, batch_size=args.batch_size, num_workers=4, shuffle=True,
    drop_last=True, pin_memory=True)

In [ ]:
num_epochs = args.num_epochs
writer = SummaryWriter('logs/{}'.format(model_tag))

for epoch in range(num_epochs):
    print('Starting epoch {}'.format(epoch))
    running_loss = train_epoch(train_loader, model, optimizer, device)

    writer.add_scalar('loss', running_loss, epoch)
    
    torch.save(model.state_dict(), os.path.join(model_save_path, f'epoch_{epoch}.pth'))

Starting epoch 0


Training:   0%|          | 0/7591 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Training - Batch Loss: 0.0002: 100%|██████████| 7591/7591 [2:50:56<00:00,  1.32s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Training - Batch Loss: 0.0002: 100%|██████████| 7591/7591 [2:50:56<00:00,  1.35s/it]


Starting epoch 1


Training - Batch Loss: 0.0001: 100%|██████████| 7591/7591 [2:50:24<00:00,  1.35s/it]


Starting epoch 2


Training - Batch Loss: 0.0161: 100%|██████████| 7591/7591 [2:50:21<00:00,  1.35s/it]
